In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import trange, tqdm

In [2]:
# DATASET
class LSTMDataset(Dataset):
    def __init__(self, data, labels, sequence_length, transform=None):
        self.data = data
        self.labels = labels
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        # X: shape (sequence_length, num_features)
        X = self.data[idx : idx + self.sequence_length]
        # y: label at the next time step
        y = self.labels[idx + self.sequence_length]
        return X, y

In [3]:
path = 'inputs/A'

# TODO: Min-max/normalize data
# TODO: Generalize to other stocks

train_loaders = [] # [dataloader1, dataloader2, ...]
test_loaders = []

for i in range(1, 16):
    data_df = pd.read_csv(f'{path}/{i}.csv', index_col=0)
    data_df.drop(columns=['timestamp'], inplace=True)

    X_df = data_df.drop(columns='label')
    y_df = data_df['label']

    X = torch.from_numpy(X_df.values).to(torch.float32)
    y = torch.from_numpy(y_df.values).to(torch.float32).type(torch.LongTensor)

    dataloader = DataLoader(LSTMDataset(X, y, sequence_length=20), batch_size=1, shuffle=True)

    train_loaders.append(dataloader)

for i in range(16, 21):
    data_df = pd.read_csv(f'{path}/{i}.csv', index_col=0)
    data_df.drop(columns=['timestamp'], inplace=True)

    X_df = data_df.drop(columns='label')
    y_df = data_df['label']

    X = torch.from_numpy(X_df.values).to(torch.float32)
    y = torch.from_numpy(y_df.values).to(torch.float32).type(torch.LongTensor)

    dataloader = DataLoader(LSTMDataset(X, y, sequence_length=20), batch_size=1, shuffle=True)

    test_loaders.append(dataloader)

print('basa')

basa


## Training

In [4]:
from LSTM import LSTM
from LSTM  import LSTMTrainer

In [5]:
# MODEL
torch.set_anomaly_enabled(True)
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu'

model = LSTM(input_size=6, hidden_size=32, num_layers=1, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

trainer = LSTMTrainer(model, train_loaders, test_loaders, optimizer, criterion, device, num_epochs=10)

C:\Users\loic4\.conda\envs\NBIStockForecast\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [6]:
# TRAINING
trainer.train()

Training:   0%|          | 0/10 [00:00<?, ?epoch/s]

Epoch 0:   0%|          | 0/15 [00:00<?, ?period/s]

Evaluating:   0%|          | 0/15 [00:00<?, ?period/s]

C:\Users\loic4\PycharmProjects\NBIStockForecast\LSTM\Trainer.py:142: UserWarning: Skipping AUROC calculation for period 11. Shape mismatch: y_true has 2 unique classes, but y_scores has 3 columns.
  warnings.warn(


Epoch 1:   0%|          | 0/15 [00:00<?, ?period/s]

KeyboardInterrupt: 

In [ ]:
# TODO: Dynamic hidden state batch sizing (cuz is source of EROOR)